In [1]:
from stemming.porter2 import stem
import os, sys
import math
import numpy as np
import scipy
from scipy import sparse
from scipy.sparse import hstack, vstack
lib_path = os.path.abspath(os.path.join('spamEmailClassifier'))
sys.path.append(lib_path)
from Preprocesser import * 
from NaiveBayes import *
from svm import *

# spamDir = '"D:\\projects\\spamEmailClassifier\\spamDataset"
# nonspamDir = "D:\\projects\\spamEmailClassifier\\nonspamDataset"
spamDir = 'spamDataset'
nonspamDir = 'nonspamDataset'

# My Naive Bayes Classifier

In [2]:
trainingSetSpamFileList, testSetSpamFileList, trainingSetNonSpamFileList, testSetNonSpamFileList = getTrainingTestSet(spamDir, nonspamDir)
trainingSpamTokenList, testSpamTokenList, trainingNonSpamTokenList, testNonSpamTokenList = getNormalizedEmailList(trainingSetSpamFileList, testSetSpamFileList, trainingSetNonSpamFileList, testSetNonSpamFileList)

In [3]:
wordList, wordDistributionInSpam, wordDistributionInNonSpam, spamProbability =  getSpamWordList(trainingSpamTokenList,trainingNonSpamTokenList, 4, 9, 3, 99)
print len(wordList)

1272


### Prediction 

In [4]:
p,x,y = predict(trainingSpamTokenList, wordDistributionInSpam, wordDistributionInNonSpam, spamProbability)
s =  sum(p) / float(len(p))
print "training set spam: ", s
p,x,y = predict(trainingNonSpamTokenList, wordDistributionInSpam, wordDistributionInNonSpam, spamProbability)
ns =  1 - sum(p) / float(len(p))
print "training set non-spam: ", ns
print "training set overall: ", s * spamProbability + ns * (1 - spamProbability)

print ""
p,x,y = predict(testSpamTokenList, wordDistributionInSpam, wordDistributionInNonSpam, spamProbability)
s =  sum(p) / float(len(p))
print "test set spam: ", s
p,x,y = predict(testNonSpamTokenList, wordDistributionInSpam, wordDistributionInNonSpam, spamProbability)
ns =  1 - sum(p) / float(len(p))
print "test set non-spam: ", ns
print "test set overall: ", s * spamProbability + ns * (1 - spamProbability)


training set spam:  0.934895833333
training set non-spam:  0.995852773458
training set overall:  0.986190944028

test set spam:  0.939708939709
test set non-spam:  0.995854063018
test set overall:  0.986954918743


In [5]:
myEmails = ["I like ml", "here is a good offer to make millions dollars from littel investment", "what are you doing"]
x = map(lambda x : getTokensFromStr(x), myEmails)
p,xs,yds = predict(x, wordDistributionInSpam, wordDistributionInNonSpam, spamProbability)
print p

[False, True, False]


# Skkit-learn Naive Bayes

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
# def readFileListAndNormalize(fileList):
#     res = []
#     for i in fileList:
#         with open(i, 'r') as file :
#             data=file.read()
#             res = res + [data]
#     return res

In [7]:
trainingSpamEmails = readFileListAndNormalize(trainingSetSpamFileList)
testSpamEmails = readFileListAndNormalize(testSetSpamFileList)
trainingNonSpamEmails = readFileListAndNormalize(trainingSetNonSpamFileList)
testNonSpamEmails = readFileListAndNormalize(testSetNonSpamFileList)

In [8]:
count_vect = CountVectorizer()

In [9]:
X_train_counts = count_vect.fit_transform(trainingSpamEmails + trainingNonSpamEmails)


In [10]:
print X_train_counts[0].shape

(1, 49240)


In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [12]:
target = map (lambda x : 1, trainingSpamEmails)
target += map(lambda x : 0, trainingNonSpamEmails)

testTarget = map (lambda x : 1, testSpamEmails)
testTarget += map(lambda x : 0, testNonSpamEmails)
print len(target), len(trainingSpamEmails) + len(trainingNonSpamEmails)

2313 2313


In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB().fit(X_train_tfidf, target)


### predict new emails

In [14]:
X_train_spam_count =  count_vect.transform(trainingSpamEmails)
X_train_nonspam_count =  count_vect.transform(trainingNonSpamEmails)
X_train_spam_tfidf =  tfidf_transformer.transform(X_train_spam_count)
X_train_nonspam_tfidf =  tfidf_transformer.transform(X_train_nonspam_count)

X_test_spam_count =  count_vect.transform(testSpamEmails)
X_test_nonspam_count =  count_vect.transform(testNonSpamEmails)
X_test_spam_tfidf =  tfidf_transformer.transform(X_test_spam_count)
X_test_nonspam_tfidf =  tfidf_transformer.transform(X_test_nonspam_count)


In [15]:
X_train_spam_predicted = clf.predict(X_train_spam_tfidf) 
print "training spam set: ", float(np.sum(X_train_spam_predicted)) / len(X_train_spam_predicted)

X_train_nonspam_predicted = clf.predict(X_train_nonspam_tfidf) 
print "training nonspam set: ", float(np.sum(X_train_nonspam_predicted)) / len(X_train_nonspam_predicted)

training spam set:  0.611979166667
training nonspam set:  0.00311041990669


In [16]:
X_test_spam_predicted = clf.predict(X_test_spam_tfidf) 
print "test spam set: ", float(np.sum(X_test_spam_predicted)) / len(X_test_spam_predicted)

X_test_nonspam_predicted = clf.predict(X_test_nonspam_tfidf) 
print "test nonspam set: ", float(np.sum(X_test_nonspam_predicted)) / len(X_test_nonspam_predicted)

test spam set:  0.582120582121
test nonspam set:  0.00248756218905


### Instead of using tfidf,  my wordList generate better result.  SHOULD FIGURE OUT tfidf

In [17]:
trainingSpamNormalizedEmails = map(lambda x : ' '.join(x), trainingSpamTokenList)
testSpamNormalizedEmails = map(lambda x : ' '.join(x), testSpamTokenList)
trainingNonSpamNormalizedEmails = map(lambda x : ' '.join(x), trainingNonSpamTokenList)
testNonSpamNormalizedEmails = map(lambda x : ' '.join(x), testNonSpamTokenList)

In [18]:
count_vect_normalized = CountVectorizer()
count_vect_normalized.vocabulary = wordList
X_train_counts_normalized = count_vect_normalized.transform(trainingSpamNormalizedEmails + trainingNonSpamNormalizedEmails)
clf_normalized = BernoulliNB().fit(X_train_counts_normalized, target)



In [19]:
X_train_spam_normalized_predicted = clf_normalized.predict(count_vect_normalized.transform(trainingSpamNormalizedEmails)) 
print "training spam set: ", float(np.sum(X_train_spam_normalized_predicted)) / len(X_train_spam_normalized_predicted)

X_train_nonspam_normalized__predicted = clf_normalized.predict(count_vect_normalized.transform(trainingNonSpamNormalizedEmails))
print "training nonspam set: ", 1- float(np.sum(X_train_nonspam_normalized__predicted)) / len(X_train_nonspam_normalized__predicted)


predict = clf_normalized.predict(count_vect_normalized.transform(testSpamNormalizedEmails)) 
print "test spam set: ", float(np.sum(predict)) / len(predict)

predict = clf_normalized.predict(count_vect_normalized.transform(testNonSpamNormalizedEmails))
print "test nonspam set: ", 1- float(np.sum(predict)) / len(predict)


training spam set:  0.880208333333
training nonspam set:  1.0
test spam set:  0.866943866944
test nonspam set:  0.999585406302


### use pipeline

In [20]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(trainingSpamNormalizedEmails + trainingNonSpamNormalizedEmails, target)  

In [21]:
predicted = text_clf.predict(trainingSpamNormalizedEmails + trainingNonSpamNormalizedEmails)
print predicted
np.mean(predicted == target)    

[0 0 0 ..., 0 0 0]


0.8430609597924773

In [22]:
from sklearn import metrics
print(metrics.classification_report(target, predicted,
    target_names=['non-spam', 'spam']))

             precision    recall  f1-score   support

   non-spam       0.84      1.00      0.91      1929
       spam       1.00      0.05      0.10       384

avg / total       0.87      0.84      0.78      2313



# Sklearn SVM 

In [23]:
from sklearn import svm
clf = svm.SVC(class_weight = 'auto', kernel = 'linear')
clf.fit(X_train_counts_normalized, target)

SVC(C=1.0, cache_size=200, class_weight='auto', coef0=0.0, degree=3,
  gamma=0.0, kernel='linear', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [24]:
predict = clf.predict(count_vect_normalized.transform(trainingSpamNormalizedEmails))
print float(sum(predict)) / len(predict)

predict = clf.predict(count_vect_normalized.transform(trainingNonSpamNormalizedEmails))
print 1- float(sum(predict)) / len(predict)


1.0
0.987558320373


In [25]:
predicted = clf.predict(count_vect_normalized.transform(testSpamNormalizedEmails + testNonSpamNormalizedEmails))
print(metrics.classification_report(testTarget, predicted,
    target_names=['non-spam', 'spam']))

             precision    recall  f1-score   support

   non-spam       1.00      0.98      0.99      2412
       spam       0.92      0.99      0.95       481

avg / total       0.98      0.98      0.98      2893



In [26]:
param_grid = [
  {'C': [1, 10, 30, 60, 90, 120, 600, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 30, 60, 90, 120, 600, 1000], 'gamma': [0.01, 0.001, 0.0001], 'kernel': ['rbf']},
 ]

scores = ['precision', 'recall']

In [27]:
from sklearn.grid_search import GridSearchCV
for score in scores:
    clf = GridSearchCV(svm.SVC(), param_grid, cv=5, scoring=score)
    clf.fit(X_train_counts_normalized, target)
    
    predict = clf.predict(count_vect_normalized.transform(trainingSpamNormalizedEmails))
    print float(sum(predict)) / len(predict)
    
    predict = clf.predict(count_vect_normalized.transform(trainingNonSpamNormalizedEmails))
    print 1- float(sum(predict)) / len(predict)



0.744791666667
1.0
0.9765625
1.0


C:\Anaconda\lib\site-packages\sklearn\metrics\metrics.py:1771: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
predict = clf.predict(count_vect_normalized.transform(trainingSpamNormalizedEmails))
print float(sum(predict)) / len(predict)

predict = clf.predict(count_vect_normalized.transform(trainingNonSpamNormalizedEmails))
print 1- float(sum(predict)) / len(predict)



0.9765625
1.0


# My SVM Implementation  # Optimization Not Working

In [4]:
def getMatrixFromNormalizedEmailList(emaiList, wordList):
    t = []
    for i in emaiList:
        tmp = [ word in i for word in wordList ]  
        t.append(tmp)
    return sparse.csr_matrix(t)


In [5]:
trainingNonSpamVectors = getMatrixFromNormalizedEmailList(trainingNonSpamEmails, wordList)
trainingSpamVectors = getMatrixFromNormalizedEmailList(trainingSpamEmails, wordList)
testSpamVectors = getMatrixFromNormalizedEmailList(testSpamEmails, wordList)
testNonSpamVectors = getMatrixFromNormalizedEmailList(testNonSpamEmails, wordList)




NameError: name 'trainingNonSpamEmails' is not defined

In [ ]:
def getLinearKernelMatrix(trainingMatrix):
    return trainingMatrix * trainingMatrix.T

In [ ]:
trainingMatrix = vstack([trainingSpamVectors, trainingNonSpamVectors])
trainingLabel = np.matrix([1 for i in trainingSpamVectors] + [-1 for i in trainingNonSpamVectors]).T

testgMatrix = vstack([testSpamVectors, testNonSpamVectors])
testLabel = np.matrix([1 for i in testSpamVectors] + [-1 for i in testNonSpamVectors]).T 

y = trainingLabel

K = getLinearKernelMatrix(trainingMatrix)

alpha0 = np.matrix([0 for i in trainingLabel]).T 

Y = np.zeros((trainingLabel.shape[0], trainingLabel.shape[0]), int)
for i in range(0, Y.shape[0]):
    for j in range(0, Y.shape[0]):
        if i == j:
            Y[i, j] = trainingLabel[i]

In [ ]:
def jacobian(alpha):
    return np.matrix([1 for i in trainingLabel]).T + Y * K * Y * alpha

In [ ]:
def optimizeObj(alpha):
    return (alpha.T * Y * K * Y.T * alpha)[0,0] + sum(alpha)

def constraint(alpha):
    return (alpha.T * y)[0,0]
print constraint(y)


In [35]:
cons = [{'type':'eq', 'fun': constraint}]

In [37]:

# print scipy.optimize.fmin_slsqp(optimizeObj, alpha0, f_ieqcons =constraint, fprime = jacobian)

#Deciscion Tree ID3

## Single Tree ID3

In [29]:
wordList, wordDistributionInSpam, wordDistributionInNonSpam, spamProbability =  getSpamWordList(trainingSpamTokenList,trainingNonSpamTokenList, 30, 2, 2, 179)

trainingSpamEmails = readFileListAndNormalize(trainingSetSpamFileList)
testSpamEmails = readFileListAndNormalize(testSetSpamFileList)
trainingNonSpamEmails = readFileListAndNormalize(trainingSetNonSpamFileList)
testNonSpamEmails = readFileListAndNormalize(testSetNonSpamFileList)

trainingNonSpamVectors = getMatrixFromNormalizedEmailList(trainingNonSpamEmails, wordList)
trainingSpamVectors = getMatrixFromNormalizedEmailList(trainingSpamEmails, wordList)
testSpamVectors = getMatrixFromNormalizedEmailList(testSpamEmails, wordList)
testNonSpamVectors = getMatrixFromNormalizedEmailList(testNonSpamEmails, wordList)

trainingMatrix = vstack([trainingSpamVectors, trainingNonSpamVectors])
trainingLabel = np.matrix([1 for i in trainingSpamVectors] + [-1 for i in trainingNonSpamVectors]).T

testgMatrix = vstack([testSpamVectors, testNonSpamVectors])
testLabel = np.matrix([1 for i in testSpamVectors] + [-1 for i in testNonSpamVectors]).T 

y = trainingLabel

In [30]:
print len(wordList), trainingMatrix.shape, trainingLabel.shape, testgMatrix.shape

145 (2313, 145) (2313L, 1L) (2893, 145)


In [31]:
trainingDataSet = trainingMatrix.toarray().tolist()
trainingLabels = [ i[0] for i in trainingLabel.tolist()]
testDataSet = testgMatrix.toarray().tolist()
testLabels = [ i[0] for i in testLabel.tolist()]

In [32]:
#import ID3Tree as id3
#id3 = reload(id3)

# dump all useful data to file
toDump = [trainingDataSet, trainingLabels, testDataSet, testLabels, wordList]
fileNames = ['trainingDataSet', 'trainingLabels', 'testDataSet', 'testLabels', 'wordList_145']
for i in range(len(toDump)):
    id3.dumpToFile(toDump[i],fileNames[i])

In [28]:
print len(testLabels)
id3.dumpToFile(wordList,fileNames[4])

0


In [19]:
trainingDataSet = grabDataFromFile('trainingDataSet')
trainingLabels = grabDataFromFile('trainingLabels')
testDataSet = grabDataFromFile('testDataSet')
testLabels = grabDataFromFile('testLabels')
wordList = grabDataFromFile('wordList_145')

In [ ]:

tree = id3.createTree(trainingDataSet, trainingLabels, wordList, len(wordList))

In [ ]:
# print tree

In [ ]:
predictList = []
for i in trainingDataSet:
    t = id3.predict(i, tree, wordList)
    if t is None:
        predictList += [ '-9' ]
    else:
        predictList += [t ]
trainingPred = [trainingLabels[i] == predictList[i] for i in range(len(predictList))]
predictList = []

for i in testDataSet:
    t = id3.predict(i, tree, wordList)
    if t is None:
        predictList += [ '-9' ]
    else:
        predictList += [t ]
testPred = [testLabels[i] == predictList[i] for i in range(len(predictList))]

In [ ]:
print 'Training set accuracy: ', float(sum(trainingPred)) / len(trainingPred)
print 'Test set accuracy: ', float(sum(testPred)) / len(testPred)

In [11]:
# print len(pred

##Random Forest

In [87]:
import ID3Tree as id3
id3 = reload(id3)

# def createForrest(dataSet, labels, numTrees, bootstrapSampleSize, numFeatures):
#     bootstrapDataSet = id3.bootstrapTrainingData(dataSet, labels, bootstrapSampleSize)
#     trees = []
#     for i in range(numTrees):
#         tree = id3.createTree(bootstrapDataSet, labels, wordList, numFeatures)
#         trees += [tree]
#     return trees
    
# def forrestPredict(emailVector, trees, wordList):
#     res = []
#     for i in trees:
#         res += [ id3.predict(emailVector, tree, wordList)  ]
#     countSpam = len(  [i for i in res if i == 1] )
#     countNonSpam = len(res) - countSpam
#     if countSpam > countNonSpam:
#         return 1
#     else:
#         return -1
    
    

    

In [90]:
trees = id3.createForrest(trainingDataSet, trainingLabels, numTrees=20, bootstrapSampleSize=500, numFeatures=80, wordList = wordList)

In [ ]:
id3.dumpToFile(trees, 'forest')

In [91]:
trainingPred = id3.getPredictionAccuracyUsingTrees(trees, trainingDataSet, trainingLabel, wordList)
testPred = id3.getPredictionAccuracyUsingTrees(trees, testDataSet, testLabel, wordList)
print 'Training set accuracy: ', trainingPred
print 'Test set accuracy: ', testPred

Training set accuracy:  0.833981841764
Test set accuracy:  0.8337366056


In [73]:
#print testPred
